## Web Crawing
- 웹 페이지에서 데이터를 수집하는 방법에 대해서 학습

### 웹크롤링 방법

#### 웹페이지의 종류
- 정적인 페이지 : 웹 브라우져에 화면이 한번 뜨면 이벤트에 의한 화면의 변경이 없는 페이지
(페이지의 데이터가 변경될 때 URL이 변경) -> HTML
- 동적인 페이지 : 웹 브라우져에 화면이 뜨고 이벤트가 발생하면 서버에서 데이터를 가져와 화면을 변경하는 페이지 (페이지의 데이터가 변경될 때 URL이 변경되지 않음) -> JSON

#### requests package (in python)
- 브라우저의 URL을 입력하면 서버에서 데이터를 다운받아 화면에 출력: URL -> Data
- requests package: URL -> Data

#### requests 이용
- 받아오는 문자열에 따라 두가지 방법으로 구분
    - json 문자열로 받아서 파싱하는 방법 : 주로 동적 페이지 크롤링할때 사용 
    - html 문자열로 받아서 파싱하는 방법 : 주로 정적 페이지 크롤링할때 사용
        
#### selenium 이용
- 브라우져를 직접 열어서 데이터를 받는 방법

#### 크롤링 방법에 따른 속도
- requests json > requests html > selenium

### Crwaling Naver Stock Datas
- 네이버 증권 사이트(https://m.stock.naver.com/)에서 주가 데이터 수집
- 수집할 데이터 : 일별 kospi, kosdaq 지수, 일별 원달러(USD) 환율(exchange rate) 데이터
- 데이터 수집 절차
    - 웹서비스 분석 : url
    - 서버에 데이터 요청 : request(url) > response : json(str)
    - 서버에서 받은 데이터 파싱(데이터 형태를 변경) : json(str) > list, dict > DataFrame

In [1]:
import pandas as pd
import requests

#### 1. 웹서비스 분석 : url
- 크롬 개발자 도구(단축키: F12) 사용하여 url 탐색
    - 네트워크 탭에서 전체 초기화 후 버튼을 눌렀을 때 새로 생기는 url 찾기
    - 해당 url의 헤더 탭에서 url, 메서드, 상태 코드 등의 정보 얻을 수 있음
    - 해당 url의 미리보기/응답 탭에서 어떤 정보를 가져올 수 있는지 볼 수 있음
- pc 웹페이지가 복잡하면 mobile 웹페이지에서 수집 -> 개발자 도구에서 Device 변경 후 새로고침
    - 새로고침 안 될경우 주소창 다시 입력 후 엔터

In [2]:
page, page_size = 1, 10 # 1 페이지의 10개 정보
url = f'https://m.stock.naver.com/api/index/KOSPI/price?pageSize={page_size}&page={page}'
print(url)

https://m.stock.naver.com/api/index/KOSPI/price?pageSize=10&page=1


#### 2. 서버에 데이터 요청 : request(url) > response : json(str)
- response의 status code가 200이 나오는지 확인
- 403이나 500이 나오면 request가 잘못되거나 web server에서 수집이 안되도록 설정이 된것임
    - header 설정 또는 selenium 사용
- 200이 나오더라도 response 안에 있는 내용을 확인 > 확인하는 방법 : response.text

In [3]:
response = requests.get(url) # url에 맞는 함수(get/post) 사용하여 객체에 저장
response

<Response [200]>

In [4]:
response.text[:200] # 리스트 내에 딕셔너리 자료형 -> string 으로 반환

'[{"localTradedAt":"2022-08-03","closePrice":"2,452.34","compareToPreviousClosePrice":"12.72","compareToPreviousPrice":{"code":"2","text":"상승","name":"RISING"},"fluctuationsRatio":"0.52","openPrice":"2'

#### 3. 서버에서 받은 데이터 파싱(데이터 형태를 변경) : json(str) > list, dict > DataFrame

In [5]:
columns = ["localTradedAt", "closePrice"]
datas = response.json() # list 자료형으로 변환 -> 딕셔너리 하나가 raw data
kospi_df = pd.DataFrame(datas)[columns] # columns 선택하지 않을 시 모든 행 포함
kospi_df.tail(2)

,localTradedAt,closePrice
18,2022-06-14,"2,492.97"
19,2022-06-13,"2,504.51"


#### 4. 함수로 만들기

In [21]:
def stock_price(code="KOSPI", page=1, page_size=60):
    """
    This function is crwaling stock price from naver stock web page
    
    parameters:
        code(str): KOSPI or KOSDAQ
        page(int): page number
        page_size(int): one page size
    
    return:
        type: DataFame(pandas) : display date, price columns
    """
    url = f'https://m.stock.naver.com/api/index/{code}/price?pageSize={page_size}&page={page}'
    response = requests.get(url)
    columns = ["localTradedAt", "closePrice"]
    datas = response.json()
    return pd.DataFrame(datas)[columns]

In [6]:
# docstring: 함수를 사용하는 방법을 문자열로 작성
# 주피터 노트북의 경우 함수명 클릭 후 shift + tab을 눌러도 같은 기능
help(stock_price)

Help on function stock_price in module __main__:

stock_price(code='KOSPI', page=1, page_size=20)
    This function is crwaling stock price from naver stock web page
    parameters :
        code(str): KOSPI or KOSDAQ
        page(int): page number
        page_size(int): one page size
    return :
        type : DataFame of pandas



In [22]:
kospi_df = stock_price() # 최근 60일간의 KOSPI 지수 수집
kospi_df.tail(2)

,localTradedAt,closePrice
58,2022-05-11,"2,592.27"
59,2022-05-10,"2,596.56"


In [23]:
# KOSDAQ 데이터 수집 코드
kosdaq_df = stock_price("KOSDAQ")
kosdaq_df.tail(2)

,localTradedAt,closePrice
58,2022-05-11,866.34
59,2022-05-10,856.14


#### 5. 원달러 환율 데이터 수집 : 실습
- https://m.stock.naver.com/ > 시작지표 > 미국 USD > 일별시세(더보기)

In [16]:
def exchage_rate(code="USD", page=1, page_size=60):
    url = f'https://api.stock.naver.com/marketindex/exchange/FX_{code}KRW/prices?page={page}\
&pageSize={page_size}'
    response = requests.get(url)
    columns = ["localTradedAt", "closePrice"] # localTradeAt: 날짜, closePrice: 종가
    datas = response.json()
    return pd.DataFrame(datas)[columns]

In [17]:
# 최근 60일치 원달러 환율 데이터 수집
usd_df = exchage_rate()
usd_df.tail(2)

,localTradedAt,closePrice
58,2022-05-11,"1,274.50"
59,2022-05-10,"1,276.00"


In [10]:
eur_df = exchage_rate("EUR")
eur_df.tail(2)

,localTradedAt,closePrice
58,2022-05-11,"1,343.77"
59,2022-05-10,"1,346.56"


#### 6. 상관관계 분석
- 피어슨 상관계수(Pearson Correlation Coefficient): df.corr()
- 두 데이터 집합의 상관도를 분석할때 사용되는 지표
- 상관계수의 해석
    - -1에 가까울수록 서로 반대방향으로 움직임
    - 1에 가까울수록 서로 같은방향으로 움직임
    - 0에 가까울수록 두 데이터는 관계가 없음
- 이외에 spearman, kendal과 같은 비선형관계도 분석 가능

In [24]:
# 해석 1 : kospi, kosdaq은 아주 강한 양의 상관관계를 갖는다. (데이터가 같은 방향으로 움직임)
# 해석 2 : kospi와 usd를 강한 음의 상관관계를 갖는다. (데이터가 반대 방향으로 움직임)

# 데이터 전처리
df = kospi_df.copy()
df["kosdaq"] = kosdaq_df["closePrice"]
df["usd"] = usd_df["closePrice"]
df.rename(columns={'closePrice': 'kospi'}, inplace=True)

,kospi,kosdaq,usd,eur
kospi,1.000000,0.991273,-0.885754,-0.500534
kosdaq,0.991273,1.000000,-0.896188,-0.541728
usd,-0.885754,-0.896188,1.000000,0.452319
eur,-0.500534,-0.541728,0.452319,1.000000


In [30]:
# column의 데이터 타입 변경: str > float
# 원본 데이터에 콤마 없을 경우 pd.to_numeric() 사용하여 타입 변경 가능

df['kospi'] = df['kospi'].apply(lambda data: float(data.replace(",", "")))
df['kosdaq'] = df['kosdaq'].apply(lambda data: float(data.replace(",", "")))
df['usd'] = df['usd'].apply(lambda data: float(data.replace(",", "")))
df.dtypes

localTradedAt     object
kospi            float64
kosdaq           float64
usd              float64
dtype: object

In [31]:
# kospi - kosdaq(0.984): 1에 가까우므로 강한 양의 상관관계
# kospi - usd(-0.878): -1에 가까우므로 강한 음의 상관관계
df[['kospi', 'kosdaq', 'usd']].corr()

,kospi,kosdaq,usd
kospi,1.000000,0.984075,-0.878489
kosdaq,0.984075,1.000000,-0.821657
usd,-0.878489,-0.821657,1.000000
